#**Thông tin sinh viên:**

* MSSV: 19521388
* Họ và tên: Hoàng Tiến Dũng
* Lớp môn học: CS331.M11
* Bài tập: Cài đặt Linear Regression và Logistic Regression bằng Numpy

#**Bài làm:**

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

#**I. Linear Regression**

In [40]:
class LinearRegression:
  def __init__(self):
    pass
  def cost_function(self, X, y, theta):
    m = len(y)
    y_pred = X@theta
    return (1/(2*m)) * np.sum((y_pred-y)**2)
  
  def gradient_descent(self, X, y, theta, iters, alpha):
    m = len(y)
    costs = []
    for i in range(iters):
      y_pred = X@theta
      theta -= (alpha/m * (X.T@(y_pred-y))) #Vector hóa
      costs.append(self.cost_function(X, y,theta))
    return theta, costs

  def fit(self, X, y, iters=15, learning_rate=0.01):
    m = len(y)
    n = len(X[0])
    self.X_train = np.append(np.ones((m,1)), X.copy(), axis=1) #X: mxn
    self.y_train = y #y : mx1
    self.theta = np.zeros((n + 1, 1)) # theta = nx1
    self.theta, self.costs = self.gradient_descent(self.X_train, self.y_train, self.theta, iters=iters, alpha=learning_rate)
    return self.theta, self.costs
  
  def predict(self, X_pred):
    X_pred_t = np.append(np.ones((len(X_pred), 1)), X_pred, axis=1)
    return X_pred_t@self.theta

#**II. Logistic Regression**

In [162]:
class LogisticRegression():
  def __init__(self):
    pass

  def sigmoid(self, z):
    return 1.0/(1.0 + np.exp(-z))

  def cost_function_1(self, X, y, theta): #cross entropy
    m = len(y)
    y_pred = self.sigmoid(X@theta)
    cost = -1.0* np.mean(y*np.log(y_pred) + (1-y)*np.log(1-y_pred))
    return cost

  def gradient_descent_1(self, X, y, theta, iters , alpha ):
    m = len(y)
    costs = []
    for i in range(iters):
      h = self.sigmoid(X@theta)
      gradient = X.T@(h-y) 
      theta -= alpha / m * gradient 
      costs.append(self.cost_function_1(X, y, theta))
    return theta, costs

  def cost_function_2(self, X, y, theta): #mean squared error
    m = len(y)
    y_pred = self.sigmoid(X@theta)
    return (1/(2*m)) * np.sum((y_pred-y)**2)

  def gradient_descent_2(self, X, y, theta, iters,alpha):
    m = len(y)
    costs = []
    for i in range(iters):
      y_pred = self.sigmoid(X@theta)
      theta -= (alpha/m * (X.T@(y_pred-y)))
      costs.append(self.cost_function_2(X, y,theta))
    return theta, costs

  def fit(self, X, y, iters=15, learning_rate=0.01, loss="crossentropy"):
    m = len(y)
    n = len(X[0])
    self.X_train = np.append(np.ones((m,1)), X, axis=1) #X: mxn
    self.y_train = y #y : mx1
    self.theta = np.zeros((n + 1, 1)) # theta = nx1
    if loss == "crossentropy":
      self.theta, self.costs = self.gradient_descent_1(self.X_train, self.y_train, self.theta, iters=iters, alpha = learning_rate)
    elif loss == 'mse':
      self.theta, self.costs = self.gradient_descent_2(self.X_train, self.y_train, self.theta, iters=iters, alpha = learning_rate)
    return self.theta, self.costs

  def predict(self, X_pred):
    X_pred_t = np.append(np.ones((len(X_pred), 1)), X_pred, axis=1)
    pred = self.sigmoid(X_pred_t@self.theta)
    return np.where(pred < 0.5, 0, 1)

#**III. Thực nghiệm**

##**1. Linear Regression**

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
df = pd.read_csv('/content/drive/MyDrive/Private_ThiGiacMayTinhNangCao/Real estate.csv', index_col=0)
df_new = df.drop(df.columns[[0,2]], axis=1)
X = df_new.iloc[:,0:4]
y = df_new.iloc[:,-1]

In [8]:
X=X.to_numpy()
y=y.to_numpy()
print(X)
print(y)

[[ 32.       10.       24.98298 121.54024]
 [ 19.5       9.       24.98034 121.53951]
 [ 13.3       5.       24.98746 121.54391]
 ...
 [ 18.8       7.       24.97923 121.53986]
 [  8.1       5.       24.96674 121.54067]
 [  6.5       9.       24.97433 121.5431 ]]
[ 37.9  42.2  47.3  54.8  43.1  32.1  40.3  46.7  18.8  22.1  41.4  58.1
  39.3  23.8  34.3  50.5  70.1  37.4  42.3  47.7  29.3  51.6  24.6  47.9
  38.8  27.   56.2  33.6  47.   57.1  22.1  25.   34.2  49.3  55.1  27.3
  22.9  25.3  47.7  46.2  15.9  18.2  34.7  34.1  53.9  38.3  42.   61.5
  13.4  13.2  44.2  20.7  27.   38.9  51.7  13.7  41.9  53.5  22.6  42.4
  21.3  63.2  27.7  55.   25.3  44.3  50.7  56.8  36.2  42.   59.   40.8
  36.3  20.   54.4  29.5  36.8  25.6  29.8  26.5  40.3  36.8  48.1  17.7
  43.7  50.8  27.   18.3  48.   25.3  45.4  43.2  21.8  16.1  41.   51.8
  59.5  34.6  51.   62.2  38.2  32.9  54.4  45.7  30.5  71.   47.1  26.6
  34.1  28.4  51.6  39.4  23.1   7.6  53.3  46.4  12.2  13.   30.6  59.6
  31.3

In [23]:
y = y.reshape((414,1))
y.shape

(414, 1)

In [41]:
linearmodel = LinearRegression()

In [42]:
theta, costs = linearmodel.fit(X, y)

In [43]:
theta

array([[1.95430851e+30],
       [3.49038840e+31],
       [8.00978389e+30],
       [4.87972149e+31],
       [2.37513677e+32]])

In [44]:
costs

[17584469.24866178,
 429467665847.9636,
 1.0488991573249486e+16,
 2.561751512226611e+20,
 6.25662702135463e+24,
 1.5280709896144735e+29,
 3.7320443448709856e+33,
 9.114861211780159e+37,
 2.226144365733322e+42,
 5.436965656351846e+46,
 1.3278831329796415e+51,
 3.2431207520905476e+55,
 7.9207514211279625e+59,
 1.934504074042235e+64,
 4.72468559296495e+68]

In [ ]:
linearmodel.predict(X)

##**2. Logistic Regression**

In [46]:
from sklearn.datasets import load_iris

In [47]:
X, y = load_iris(return_X_y=True)

In [48]:
X.shape

(150, 4)

In [50]:
y = y.reshape((len(y),1))
y.shape

(150, 1)

In [113]:
y = np.where(y==1, 0, 1)

In [114]:
y

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [173]:
logisticmodel = LogisticRegression()

In [174]:
theta1, costs1 = logisticmodel.fit(X, y, iters=20, learning_rate=0.1, loss='crossentropy')

In [175]:
theta1

array([[ 0.02402407],
       [ 0.09978285],
       [ 0.24385102],
       [-0.16425354],
       [-0.02256805]])

In [176]:
costs1

[0.6513035021480682,
 0.637076241590133,
 0.6321563490621346,
 0.6283492430805254,
 0.6250014727726031,
 0.6219618853913907,
 0.619184044094222,
 0.616639294880123,
 0.6143041108143823,
 0.6121578240190163,
 0.6101819988458836,
 0.608360152170229,
 0.6066775271197683,
 0.6051209036987233,
 0.6036784290689496,
 0.6023394677125302,
 0.6010944681216385,
 0.5999348446500585,
 0.5988528727000604,
 0.5978415958111765]

In [178]:
logisticmodel.predict(X) #Vì data ít nên kết quả chưa tốt

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [163]:
logisticmodel = LogisticRegression()

In [179]:
theta2, costs2 = logisticmodel.fit(X, y, iters=15, loss='mse') #Sử dụng mean squared error cho logistic regression

In [165]:
theta2

array([[0.0113224 ],
       [0.05995518],
       [0.0490309 ],
       [0.01435911],
       [0.00595626]])

In [166]:
costs2

[0.12149171918419262,
 0.11896663508204341,
 0.11714160476380528,
 0.11581203805309297,
 0.11483269133525843,
 0.11410129993055691,
 0.11354604413266882,
 0.11311644339551992,
 0.11277691729897525,
 0.11250229186633977,
 0.11227467983242104,
 0.11208131591011916,
 0.11191305134245298,
 0.1117633029409121,
 0.11162731598477872]

In [ ]:
logisticmodel.predict(X)